In [ ]:
문제 풀기: ~ 15:20

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [2]:
df = pd.read_csv('./test/card_cust.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           1000 non-null   int64  
 1   BALANCE                           1000 non-null   float64
 2   BALANCE_FREQUENCY                 1000 non-null   float64
 3   PURCHASES                         1000 non-null   float64
 4   ONEOFF_PURCHASES                  1000 non-null   float64
 5   INSTALLMENTS_PURCHASES            1000 non-null   float64
 6   CASH_ADVANCE                      1000 non-null   float64
 7   PURCHASES_FREQUENCY               1000 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        1000 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  1000 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            1000 non-null   float64
 11  CASH_ADVANCE_TRX                  1000 non-null   float64
 12  PURCHAS

# 전처리

In [3]:
base=df.copy()
base['MINIMUM_PAYMENTS']=base['MINIMUM_PAYMENTS'].fillna(base['MINIMUM_PAYMENTS'].mean())

In [4]:
base['MINIMUM_PAYMENTS'].isna().sum()

0

# Q1

In [6]:
q1=base.groupby('TENURE')[['BALANCE', 'CREDIT_LIMIT']].corr()
q1

BALANCE  CREDIT_LIMIT
TENURE                                     
6.0    BALANCE       1.000000      0.868056
       CREDIT_LIMIT  0.868056      1.000000
7.0    BALANCE       1.000000      0.948405
       CREDIT_LIMIT  0.948405      1.000000
8.0    BALANCE       1.000000      0.820696
       CREDIT_LIMIT  0.820696      1.000000
9.0    BALANCE       1.000000      0.085474
       CREDIT_LIMIT  0.085474      1.000000
10.0   BALANCE       1.000000      0.291482
       CREDIT_LIMIT  0.291482      1.000000
11.0   BALANCE       1.000000      0.380360
       CREDIT_LIMIT  0.380360      1.000000
12.0   BALANCE       1.000000      0.460833
       CREDIT_LIMIT  0.460833      1.000000

In [9]:
q1.xs('BALANCE', level=1)['CREDIT_LIMIT'].round(2).max()

0.95

# Q2

In [12]:
list1=base.columns.drop('CUST_ID')
# step1=base.drop(columns='CUST_ID')
step1=base.copy()

In [11]:
len(step1.columns)

17

In [13]:
zscore=StandardScaler().fit(step1[list1])
# step1_z=pd.DataFrame(zscore.transform(step1), 

step1[list1] = zscore.transform(step1[list1])

In [14]:
step1.head(2)

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,10001,-0.848768,-0.419879,-0.441936,-0.374048,-0.395301,-0.482354,-0.872701,-0.804321,-0.719620,-0.684701,-0.457918,-0.564116,-1.161669,-0.557396,-0.443725,-0.465544,0.282429
1,10002,0.282791,0.012131,-0.469017,-0.374048,-0.470304,1.878468,-1.282558,-0.804321,-0.924403,0.513493,0.065417,-0.628057,0.150025,0.360574,-0.086159,0.331592,0.282429


In [15]:
sil_col=[]
for i in range(2,6):
    km=KMeans(n_clusters=i, random_state=1234).fit(step1[list1])
    sil=silhouette_score(step1[list1], km.labels_)
    sil_col.append([i, sil])

sil_col=pd.DataFrame(sil_col, columns=['k','sil'])

In [19]:
best_k=sil_col.loc[sil_col.sil.idxmax(),'k']
best_k

2

In [20]:
km=KMeans(n_clusters=best_k, random_state=1234).fit(step1[list1])
km.labels_

array([0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,

In [21]:
step3=base.copy()
step3['group'] = km.labels_

In [23]:
step3.groupby('group')['ONEOFF_PURCHASES'].mean().round(2).max()

2156.47

# Q3

In [24]:
train=base[(base.CUST_ID % 4) != 0].reset_index(drop=True)
test=base[(base.CUST_ID % 4) == 0].reset_index(drop=True)

In [25]:
q3_list=train.columns.drop(['CUST_ID', 'ONEOFF_PURCHASES'])

In [26]:
dt=DecisionTreeRegressor(random_state=1234).fit(train[q3_list], train['ONEOFF_PURCHASES'])
pred=dt.predict(test[q3_list])

In [27]:
from sklearn.metrics import mean_squared_error

In [28]:
round(mean_squared_error(test['ONEOFF_PURCHASES'], pred) ** 0.5, 2)

1039.19